In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
file_train_data = "data/train.csv"
file_test_data = "data/test.csv"

In [2]:
import csv


train_csv = []
with open(file_train_data, "r") as f:
    reader = csv.DictReader(f)

    for row in reader:
        train_csv.append(row)



test_csv = []
with open(file_test_data, "r") as f:
    reader = csv.DictReader(f)

    for row in reader:
        test_csv.append(row)

print(f"Train data size: {len(train_csv)}")
print(train_csv[:3])
print(f"Columns ({len(train_csv[0].keys())}):", train_csv[0].keys())

print(f"Test data size: {len(test_csv)}")
print(test_csv[:3])
print(f"Columns: ({len(test_csv[0].keys())})", test_csv[0].keys())

Train data size: 1460
[{'Id': '1', 'MSSubClass': '60', 'MSZoning': 'RL', 'LotFrontage': '65', 'LotArea': '8450', 'Street': 'Pave', 'Alley': 'NA', 'LotShape': 'Reg', 'LandContour': 'Lvl', 'Utilities': 'AllPub', 'LotConfig': 'Inside', 'LandSlope': 'Gtl', 'Neighborhood': 'CollgCr', 'Condition1': 'Norm', 'Condition2': 'Norm', 'BldgType': '1Fam', 'HouseStyle': '2Story', 'OverallQual': '7', 'OverallCond': '5', 'YearBuilt': '2003', 'YearRemodAdd': '2003', 'RoofStyle': 'Gable', 'RoofMatl': 'CompShg', 'Exterior1st': 'VinylSd', 'Exterior2nd': 'VinylSd', 'MasVnrType': 'BrkFace', 'MasVnrArea': '196', 'ExterQual': 'Gd', 'ExterCond': 'TA', 'Foundation': 'PConc', 'BsmtQual': 'Gd', 'BsmtCond': 'TA', 'BsmtExposure': 'No', 'BsmtFinType1': 'GLQ', 'BsmtFinSF1': '706', 'BsmtFinType2': 'Unf', 'BsmtFinSF2': '0', 'BsmtUnfSF': '150', 'TotalBsmtSF': '856', 'Heating': 'GasA', 'HeatingQC': 'Ex', 'CentralAir': 'Y', 'Electrical': 'SBrkr', '1stFlrSF': '856', '2ndFlrSF': '854', 'LowQualFinSF': '0', 'GrLivArea': '1710

In [3]:
remove_columns = ["MoSold", "YrSold"]
prediction_columns = ["SalePrice"]

In [4]:
# Build normalization mappings from datasets (train and test)
# This function analyzes combined data to produce:
# - data_type: Number vs Categorical per column
# - categorical_mappings: dict of {column: {value: int}}
# - keys_minmax: dict of {column: {"min": float, "max": float}}

def build_normalization_maps(datasets, remove_columns=None, prediction_columns=None):
    """
    datasets: list of lists-of-dicts (e.g., [train_csv, test_csv])
    returns: dict with keys: data_type, categorical_mappings, keys_minmax, remove_columns, prediction_columns
    Note: prediction_columns are INCLUDED in the returned mappings so you can
    denormalize predictions later (e.g., convert 0-1 output back to SalePrice).
    """
    remove_columns = remove_columns or ["MoSold", "YrSold"]
    prediction_columns = prediction_columns or ["SalePrice"]

    # Combine and shallow-copy rows; only remove explicitly unwanted columns
    combined = []
    for ds in datasets:
        for row in ds:
            r = row.copy()
            for col in remove_columns:
                if col in r:
                    del r[col]
            combined.append(r)

    if len(combined) == 0:
        return {
            "data_type": {},
            "categorical_mappings": {},
            "keys_minmax": {},
            "remove_columns": remove_columns,
            "prediction_columns": prediction_columns,
        }

    # Build the set of all keys present in any row (union), then exclude Id and remove_columns
    keys = set()
    for r in combined:
        keys.update(r.keys())
    keys.discard("Id")
    for col in remove_columns:
        keys.discard(col)
    keys = sorted(keys)

    # Helper to detect numeric values (allow floats and ints)
    def looks_like_number(s: str) -> bool:
        if s is None:
            return False
        if s == "" or s == "NA":
            return False
        try:
            float(s)
            return True
        except Exception:
            return False

    # Infer data types
    data_type = {}
    for k in keys:
        values = [r.get(k, "") for r in combined]
        unique_values = set(values)
        if all(looks_like_number(v) or v in ("", "NA") for v in unique_values):
            data_type[k] = "Number"
        else:
            data_type[k] = "Categorical"

    # Create categorical mappings
    categorical_mappings = {}
    for k, t in data_type.items():
        if t == "Categorical":
            unique_values = list(sorted(set(r.get(k, "") for r in combined)))
            mapping = {v: i for i, v in enumerate(unique_values)}
            categorical_mappings[k] = mapping

    # Compute min/max for all keys (after applying categorical mapping for categorical columns)
    keys_minmax = {}
    for k in keys:
        vals = []
        for d in combined:
            v = d.get(k, "")
            if data_type[k] == "Number":
                if v == 'NA' or v == '':
                    vals.append(0.0)
                else:
                    try:
                        vals.append(float(v))
                    except Exception:
                        vals.append(0.0)
            else:
                # categorical -> integer via mapping
                vals.append(categorical_mappings[k].get(v, 0))
        # Guard against empty vals (shouldn't happen because keys were collected from combined)
        if len(vals) == 0:
            keys_minmax[k] = {"min": 0.0, "max": 1.0}
        else:
            keys_minmax[k] = {"min": float(min(vals)), "max": float(max(vals))}

    return {
        "data_type": data_type,
        "categorical_mappings": categorical_mappings,
        "keys_minmax": keys_minmax,
        "remove_columns": remove_columns,
        "prediction_columns": prediction_columns,
    }


# Build combined mappings using both train and test CSVs (so both datasets share the same normalization)
mappings = build_normalization_maps([train_csv, test_csv], remove_columns=remove_columns, prediction_columns=prediction_columns)

mappings

{'data_type': {'1stFlrSF': 'Number',
  '2ndFlrSF': 'Number',
  '3SsnPorch': 'Number',
  'Alley': 'Categorical',
  'BedroomAbvGr': 'Number',
  'BldgType': 'Categorical',
  'BsmtCond': 'Categorical',
  'BsmtExposure': 'Categorical',
  'BsmtFinSF1': 'Number',
  'BsmtFinSF2': 'Number',
  'BsmtFinType1': 'Categorical',
  'BsmtFinType2': 'Categorical',
  'BsmtFullBath': 'Number',
  'BsmtHalfBath': 'Number',
  'BsmtQual': 'Categorical',
  'BsmtUnfSF': 'Number',
  'CentralAir': 'Categorical',
  'Condition1': 'Categorical',
  'Condition2': 'Categorical',
  'Electrical': 'Categorical',
  'EnclosedPorch': 'Number',
  'ExterCond': 'Categorical',
  'ExterQual': 'Categorical',
  'Exterior1st': 'Categorical',
  'Exterior2nd': 'Categorical',
  'Fence': 'Categorical',
  'FireplaceQu': 'Categorical',
  'Fireplaces': 'Number',
  'Foundation': 'Categorical',
  'FullBath': 'Number',
  'Functional': 'Categorical',
  'GarageArea': 'Number',
  'GarageCars': 'Number',
  'GarageCond': 'Categorical',
  'GarageFi

In [5]:
import copy
import itertools

def parse_data(lines, mappings=None):
    """Parse rows (list of dicts) into normalized train/test examples.

    If `mappings` is provided (as returned by `build_normalization_maps`),
    it'll be used for categorical mapping and numeric min/max normalization.
    Otherwise mappings will be inferred from the single `lines` dataset.
    """
    # Make a shallow copy of the data so we don't mutate the original list
    data = [r.copy() for r in lines]

    # Remove unwanted columns
    for r in data:
        for col in remove_columns:
            if col in r:
                del r[col]

    # If no mappings provided, infer from this dataset alone
    if mappings is None:
        mappings = build_normalization_maps([data], remove_columns=remove_columns, prediction_columns=prediction_columns)

    data_type_analysis = mappings["data_type"]
    normalized_categorial_mappings = mappings.get("categorical_mappings", {})
    keys_minmax = mappings.get("keys_minmax", {})

    # Transform data according to type
    for l in data:
        for k in list(l.keys()):
            if k == "Id":
                continue

            if data_type_analysis.get(k) == "Number":
                if l[k] == 'NA' or l[k] == '':
                    l[k] = 0.0
                else:
                    try:
                        l[k] = float(l[k])
                    except Exception:
                        # fallback
                        l[k] = 0.0
            else:
                # Map categorical values to integers using provided mapping
                mapping = normalized_categorial_mappings.get(k, {})
                l[k] = mapping.get(l[k], 0)

    # Normalize using keys_minmax (0..1)
    normalized_data = copy.deepcopy(data)
    keys = [k for k in normalized_data[0].keys()]

    for r in normalized_data:
        for k in keys:
            if k == "Id":
                continue
            value = float(r[k])
            minmax = keys_minmax.get(k, {"min": 0.0, "max": 1.0})
            denom = (minmax["max"] - minmax["min"])
            norm_value = (value - minmax["min"]) / denom if denom != 0 else 0.0
            r[k] = norm_value

    # Split into input/output structure
    train_data = []
    for r in normalized_data:
        train_data.append(
            {
                "id": r["Id"],
                "input": list(
                    {k: v for k, v in r.items() if k not in list(itertools.chain(*[prediction_columns, ["Id"]]))}.values()
                ),
                "output": [r.get(c, 0.0) for c in prediction_columns],
            }
        )

    return train_data


# Parse data using the shared mappings
train_data = parse_data(train_csv, mappings=mappings)
print(train_data[:3])

test_data = parse_data(test_csv, mappings=mappings)
print(test_data[:3])

[{'id': '1', 'input': [0.23529411764705882, 0.8, 0.20766773162939298, 0.03341980415527355, 1.0, 0.5, 1.0, 1.0, 0.0, 1.0, 0.0, 0.20833333333333334, 0.25, 0.2857142857142857, 0.0, 0.7142857142857143, 0.6666666666666666, 0.5, 0.9492753623188406, 0.8833333333333333, 0.2, 0.14285714285714285, 0.8666666666666667, 0.875, 0.25, 0.1225, 0.6666666666666666, 1.0, 0.4, 0.5, 1.0, 1.0, 0.3333333333333333, 0.12508858965272857, 1.0, 0.0, 0.0642123287671233, 0.14009819967266776, 0.2, 0.0, 1.0, 1.0, 0.10964083175803403, 0.4135593220338983, 0.0, 0.2592313489073097, 0.3333333333333333, 0.0, 0.5, 0.5, 0.375, 0.3333333333333333, 0.5, 0.46153846153846156, 1.0, 0.0, 0.6, 0.16666666666666666, 0.9075668328047123, 0.6666666666666666, 0.4, 0.3682795698924731, 1.0, 1.0, 1.0, 0.0, 0.08221024258760108, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.25, 0.0, 1.0, 0.8], 'output': [0.276158940397351]}, {'id': '2', 'input': [0.0, 0.8, 0.25559105431309903, 0.038795017411016854, 1.0, 0.5, 1.0, 1.0, 0.0, 0.5, 0.0, 1.0, 0.125, 0.285714285

In [5]:
mappings

{'data_type': {'1stFlrSF': 'Number',
  '2ndFlrSF': 'Number',
  '3SsnPorch': 'Number',
  'Alley': 'Categorical',
  'BedroomAbvGr': 'Number',
  'BldgType': 'Categorical',
  'BsmtCond': 'Categorical',
  'BsmtExposure': 'Categorical',
  'BsmtFinSF1': 'Number',
  'BsmtFinSF2': 'Number',
  'BsmtFinType1': 'Categorical',
  'BsmtFinType2': 'Categorical',
  'BsmtFullBath': 'Number',
  'BsmtHalfBath': 'Number',
  'BsmtQual': 'Categorical',
  'BsmtUnfSF': 'Number',
  'CentralAir': 'Categorical',
  'Condition1': 'Categorical',
  'Condition2': 'Categorical',
  'Electrical': 'Categorical',
  'EnclosedPorch': 'Number',
  'ExterCond': 'Categorical',
  'ExterQual': 'Categorical',
  'Exterior1st': 'Categorical',
  'Exterior2nd': 'Categorical',
  'Fence': 'Categorical',
  'FireplaceQu': 'Categorical',
  'Fireplaces': 'Number',
  'Foundation': 'Categorical',
  'FullBath': 'Number',
  'Functional': 'Categorical',
  'GarageArea': 'Number',
  'GarageCars': 'Number',
  'GarageCond': 'Categorical',
  'GarageFi

In [6]:
import torch.nn as nn
import torch
import numpy as np
from typing import List

# Determine the best available device
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")


# DEVICE = torch.device("cpu")
DEVICE = get_device()
print(DEVICE)

class SimpleNeuralNetwork(nn.Module):
    """Simple feedforward neural network using PyTorch"""

    def __init__(
        self,
        input_size: int = 16,
        hidden_layers: List[int] = [256],
        output_size: int = 4,
        empty: bool = False,
    ):
        super().__init__()

        if empty:
            return

        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size

        # Build layers using PyTorch modules
        layers = []
        prev_size = input_size

        # Add hidden layers
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            # layers.append(nn.Sigmoid())
            prev_size = hidden_size

        # Add output layer (no activation)
        layers.append(nn.Linear(prev_size, output_size))

        self.network = nn.Sequential(*layers)

        # Initialize weights using He initialization
        self._initialize_weights()

        # Move to device
        self.to(DEVICE)

    def _initialize_weights(self):
        """Initialize weights using He initialization"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight, nonlinearity="tanh")
                nn.init.zeros_(module.bias)

    def forward(self, x):
        """Forward pass through the network"""
        # Convert numpy array to tensor if needed and move to device
        if isinstance(x, np.ndarray):
            x = torch.from_numpy(x).float().to(DEVICE)
        elif isinstance(x, torch.Tensor):
            x = x.to(DEVICE)
        elif isinstance(x, list):
            x = torch.tensor(x, dtype=torch.float32).to(DEVICE)

        return self.network(x)

    def mutate(self, mutation_rate: float = 0.1, mutation_strength: float = 0.5):
        """Mutate the network's weights and biases"""
        with torch.no_grad():
            for param in self.parameters():
                if torch.rand(1).item() < mutation_rate:
                    mutation = torch.randn_like(param) * mutation_strength
                    param.add_(mutation)

cuda


In [7]:
from joblib import Parallel, delayed
import joblib
from typing import List
from typing import Tuple
import numpy as np
import random


class EvolutionaryOptimizer:
    def __init__(
        self,
        population_size: int = 50,
        elite_size: int = 10,
        new_members: int = 0,
        mutation_rate: float = 0.1,
        mutation_strength: float = 0.5,
        input_size: int = 8,
        hidden_layers: List[int] = [32],
        output_size: int = 1,
        train_data: List = [],
        train_size: int = 0,
    ):
        self.population_size = population_size
        self.elite_size = elite_size
        self.new_members = new_members
        self.mutation_rate = mutation_rate
        self.mutation_strength = mutation_strength
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layers = hidden_layers
        self.train_data = train_data
        self.train_size = train_size if train_size > 0 else len(train_data)

        # Convert training data to tensors once
        self.input_tensor = torch.tensor([data['input'] for data in train_data], dtype=torch.float32).to(DEVICE)
        self.output_tensor = torch.tensor([data['output'] for data in train_data], dtype=torch.float32).to(DEVICE)

        # Create initial population
        self.population = []
        for _ in range(population_size):
            network = SimpleNeuralNetwork(
                input_size=input_size,
                output_size=output_size,
                hidden_layers=hidden_layers,
            )
            self.population.append(network)

    def evaluate(
        self,
    ) -> List[Tuple[SimpleNeuralNetwork, int, float]]:
        if DEVICE.type == "cuda" or DEVICE.type == "mps":
            # GPU evaluation - process all networks sequentially since GPU handles parallelization
            results = []
            for network in self.population:
                network.eval()
                with torch.no_grad():
                    predictions = network(self.input_tensor)
                    predictions = (predictions > 0.5).float()
                    accuracy = (predictions == self.output_tensor).float().mean().item()
                    results.append((network, accuracy))
            return results
        else:
            # CPU evaluation - use joblib for parallel processing
            def eval_network(network: SimpleNeuralNetwork):
                network.eval()
                scores = []
                with torch.no_grad():
                    for test in self.train_data:
                        input_values = test['input']
                        prediction = network.forward(input_values)
                        prediction = True if prediction[0] > .5 else False
                        reality = True if test['output'][0] == 1.0 else False
                        scores.append(1 if prediction == reality else 0)
                return (network, sum(scores) / len(scores))

            results = Parallel(n_jobs=joblib.cpu_count())(
                delayed(eval_network)(net) for net in self.population
            )
            return results

    def select_and_breed(
        self, evaluated: List[Tuple[SimpleNeuralNetwork, int, float]]
    ) -> None:
        # Sort by score descending
        evaluated.sort(key=lambda x: x[1], reverse=True)
        elite = evaluated[: self.elite_size]

        new_population = []
        # Keep elite networks
        for net, _ in elite:
            new_population.append(net)

        # Create offspring by mutating elite networks
        while len(new_population) < self.population_size:
            parent = random.choice(elite)[0]

            # Create a child by copying the parent's state
            child = copy.deepcopy(parent)

            # Mutate the child
            child.mutate(self.mutation_rate, self.mutation_strength)
            new_population.append(child)

        # Add random new members
        for _ in range(self.new_members):
            network = SimpleNeuralNetwork(hidden_layers=self.hidden_layers, input_size=self.input_size, output_size=self.output_size)
            new_population.append(network)

        self.population = new_population

    def run_generation(
        self,
    ) -> Tuple[List[SimpleNeuralNetwork], float]:
        evaluated = self.evaluate()
        best_precision = max(precision for _, precision in evaluated)

        self.select_and_breed(evaluated)

        return self.population, best_precision

In [8]:
import pickle

def save_network(network: SimpleNeuralNetwork, filename: str):
    torch.save(network.state_dict(), filename)


def load_network(filename: str, hidden_layers: List[int]) -> SimpleNeuralNetwork:
    network = SimpleNeuralNetwork(hidden_layers=hidden_layers)
    network.load_state_dict(torch.load(filename, map_location=DEVICE))
    network.to(DEVICE)
    return network


def save_population(population: List[SimpleNeuralNetwork], filename: str):
    with open(filename, "wb") as f:
        pickle.dump(population, f)


def load_population(filename: str) -> List[SimpleNeuralNetwork]:
    with open(filename, "rb") as f:
        population = pickle.load(f)
    return population

In [9]:
import time
from datetime import timedelta

generations = 1000
hidden_layers = [64, 32, 16]
mutation_rate = 0.25
mutation_strength = 0.25

folder = f"networks/{'_'.join(str(x) for x in hidden_layers)}"
os.makedirs(folder, exist_ok=True)
print(f"Saving networks to folder: {folder}")

scores_best = []

def mutation_decay(x, mu0=0.25, k=3.0):
    return mu0 * np.exp(-k * x)

start_time = time.time()

epoch = 0
last_precision = 0
while True:
    print(f"Epoch {epoch+1} starting...")
    epoch += 1

    optimizer = EvolutionaryOptimizer(
        population_size=100,
        elite_size=10,
        new_members=0,
        mutation_rate=mutation_rate,
        mutation_strength=mutation_strength,
        input_size=len(train_data[0]['input']),
        hidden_layers=hidden_layers,
        output_size=1,
        train_data=random.sample(train_data, len(train_data) // 4),
    )

    for gen in range(generations):
        optimizer.mutation_strength = mutation_decay(
            gen / generations, k=8, mu0=mutation_strength
        )
        optimizer.mutation_rate = mutation_decay(gen / generations, k=8, mu0=mutation_rate)

        (population, best_precision) = optimizer.run_generation()
        # print(f"Epoch: {epoch} {gen+1}/{generations} - Best Precision: {best_precision:.4f}")

    elapsed_time = time.time() - start_time
    duration = str(timedelta(seconds=(elapsed_time)))
    print(f"Epoch {epoch} completed after {duration}")
    if best_precision > last_precision:
        save_population(optimizer.population, f"{folder}/population_epoch_{epoch}.pkl")
        last_precision = best_precision    
        print(f"Saved population for epoch {epoch} with precision {best_precision:.4f}")

Saving networks to folder: networks/64_32_16
Epoch 1 starting...
Epoch 1 completed after 0:01:11.587968
Saved population for epoch 1 with precision 0.0027
Epoch 2 starting...
Epoch 2 completed after 0:02:23.016937
Epoch 3 starting...
Epoch 3 completed after 0:03:35.292214
Epoch 4 starting...
Epoch 4 completed after 0:04:48.562020
Epoch 5 starting...
Epoch 5 completed after 0:06:02.289200
Epoch 6 starting...
Epoch 6 completed after 0:07:15.094142
Epoch 7 starting...
Epoch 7 completed after 0:08:26.203814
Epoch 8 starting...
Epoch 8 completed after 0:09:38.608707
Epoch 9 starting...
Epoch 9 completed after 0:10:52.539661
Epoch 10 starting...
Epoch 10 completed after 0:12:06.212588
Epoch 11 starting...
Epoch 11 completed after 0:13:19.787621
Epoch 12 starting...
Epoch 12 completed after 0:14:31.701286
Epoch 13 starting...
Epoch 13 completed after 0:15:43.432161
Epoch 14 starting...
Epoch 14 completed after 0:16:56.644674
Epoch 15 starting...
Epoch 15 completed after 0:18:08.646176
Epoch 1

KeyboardInterrupt: 

In [8]:
# Generate submission files for all saved populations
for dirname, _, filenames in os.walk(folder):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        population = load_population(os.path.join(dirname, filename))
        network = population[0]

        checkpoint = filename.split("_")[-1].split(".")[0]
        print(f"Generating submission for checkpoint {checkpoint}...")

        predictions = []
        for test in test_data:
            input_values = test['input']
            prediction = network.forward(input_values)
            # prediction = prediction.cpu()
            prediction = True if prediction[0] > .5 else False
            # print(f"Passenger ID: {test['id']}, Survived: {1 if prediction else 0}")
            predictions.append([test['id'], 1 if prediction else 0])

        # Save predictions to CSV
        with open(f"submission_{checkpoint}.csv", "w", newline="") as csvfile:
            fieldnames = ["PassengerId", "Survived"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for pid, survived in predictions:
                writer.writerow({"PassengerId": pid, "Survived": survived})



# population = load_population('networks/256/population_epoch_228.pkl')
# network = population[0]

networks/16_8_4_2/population_epoch_1.pkl
Generating submission for checkpoint 1...


In [ ]:
# Select network from the optimizer
network = optimizer.population[0]

In [ ]:
predictions = []
for test in test_data:
    input_values = test['input']
    prediction = network.forward(input_values)
    # prediction = prediction.cpu()
    prediction = True if prediction[0] > .5 else False
    # print(f"Passenger ID: {test['id']}, Survived: {1 if prediction else 0}")
    predictions.append([test['id'], 1 if prediction else 0])

# Save predictions to CSV
with open("submission.csv", "w", newline="") as csvfile:
    fieldnames = ["PassengerId", "Survived"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for pid, survived in predictions:
        writer.writerow({"PassengerId": pid, "Survived": survived})


In [ ]:
mutation_decay(1, mu0=0.25, k=8.0)

np.float64(8.386565697562796e-05)